In [397]:
import numpy as np 
import pandas as pd
import ast
import os

movies = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')


In [398]:
movies = movies.merge(credits, on='title')

- genres
- id
- keywords
- title
- overview 
- cast
- crew

In [399]:
movies = movies[['genres', 'movie_id', 'keywords', 'title', 'overview', 'cast', 'crew']]

movies.head(1).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genres    1 non-null      object
 1   movie_id  1 non-null      int64 
 2   keywords  1 non-null      object
 3   title     1 non-null      object
 4   overview  1 non-null      object
 5   cast      1 non-null      object
 6   crew      1 non-null      object
dtypes: int64(1), object(6)
memory usage: 184.0+ bytes


In [400]:
movies.isnull().sum()

genres      0
movie_id    0
keywords    0
title       0
overview    3
cast        0
crew        0
dtype: int64

In [401]:
movies.dropna(inplace=True)

In [402]:
movies.duplicated().sum()

0

In [403]:
def getNames(obj):
    endList = []
    for i in ast.literal_eval(obj):
        name = i['name'].replace(" ","")
        endList.append(name)
    return endList

movies['keywords'] = movies['keywords'].apply(getNames)
movies['genres'] = movies['genres'].apply(getNames)
movies['cast'] = movies['cast'].apply(getNames)

In [404]:
def getDirector(obj):
    director = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            name = i['name'].replace(" ","")
            director.append(name)
            break
    return director

movies['crew'] = movies['crew'].apply(getDirector)

In [405]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [406]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['crew'] 

In [407]:
cleaned_movies = movies[['movie_id', 'title', 'tags']]

In [408]:
cleaned_movies['tags'] = cleaned_movies['tags'].apply(lambda x: " ".join(x))
cleaned_movies['tags'] = cleaned_movies['tags'].apply(lambda x: x.lower())

/tmp/ipykernel_33/1641915645.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_movies['tags'] = cleaned_movies['tags'].apply(lambda x: " ".join(x))
/tmp/ipykernel_33/1641915645.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_movies['tags'] = cleaned_movies['tags'].apply(lambda x: x.lower())


In [409]:
cleaned_movies['tags'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction samworthington zoesaldana sigourneyweaver stephenlang michellerodriguez giovanniribisi joeldavidmoore cchpounder wesstudi lazalonso dileeprao mattgerald seananthonymoran jasonwhyte scottlawrence kellykilgour jamespatrickpitt seanpatrickmurphy peterdillon kevindorman kelsonhenderson davidvanhorn jacobtomuri michaelblain-rozgay joncurry lukehawker woodyschultz petermensah soniayee jahnelcurfman ilramchoi kylawarren lisaroumain debrawilson chrismala taylorkibby jodielandau julielamm cullenb.madden josephbradymadden frankietorres austinwilson sarawilson tamicawashington-miller lucybriant nathanmeister gerryblair matthewchamberlain paulyates wraywilson jamesgaylyn melvinlenoclarkiii carvonfutrell brandonjelkes micahmoch hanniyahmuhammad christophernolen christaoliver aprilmarie

In [410]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [411]:
vectors = cv.fit_transform(cleaned_movies['tags']).toarray()

In [412]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [413]:
import nltk

In [414]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [415]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [416]:
cleaned_movies['tags'].apply(stem)

0       in the 22nd century, a parapleg marin is dispa...
1       captain barbossa, long believ to be dead, ha c...
2       a cryptic messag from bond’ past send him on a...
3       follow the death of district attorney harvey d...
4       john carter is a war-weary, former militari ca...
                              ...                        
4804    el mariachi just want to play hi guitar and ca...
4805    a newlyw couple' honeymoon is upend by the arr...
4806    "signed, sealed, delivered" introduc a dedic q...
4807    when ambiti new york attorney sam is sent to s...
4808    ever sinc the second grade when he first saw h...
Name: tags, Length: 4806, dtype: object

In [417]:
from sklearn.metrics.pairwise import cosine_similarity

In [418]:
similarity = cosine_similarity(vectors)

In [419]:
cleaned_movies.head(1)

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."


In [420]:
def recommend(movie):
    movie_index = cleaned_movies[cleaned_movies['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key = lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(cleaned_movies.iloc[i[0]].title)

In [421]:
recommend('Titanic')

The Switch
Captain Phillips
The Notebook
Under the Same Moon
Ghost Ship
